In [ ]:
#!/usr/bin/env python3
import os, cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512          # 448–640 depending on speed/accuracy needs
CONF     = 0.30
IOU      = 0.45
ALPHA    = 0.40         # red rail overlay alpha

# --- color-filter (from earlier prompt) ---
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]  # in RGB
TOLERANCE         = 20.0                        # Euclidean distance in BGR space
MIN_REGION_SIZE   = 50                          # px (connected component area)
MIN_REGION_HEIGHT = 150                         # px (connected component bbox height)

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# Color+size filter (fast version of your highlight_rails_mask_only_timed)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    h, w = img_bgr.shape[:2]

    # RGB -> BGR (OpenCV is BGR)
    targets_bgr = [(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb]

    # Vectorized color distance check
    img_f = img_bgr.astype(np.float32)
    color_mask = np.zeros((h, w), dtype=bool)
    for tb in targets_bgr:
        tb_arr = np.array(tb, dtype=np.float32).reshape((1, 1, 3))
        dist = np.linalg.norm(img_f - tb_arr, axis=2)
        color_mask |= (dist <= tolerance)

    combined = rail_mask & color_mask

    # Connected component filtering
    comp_u8 = combined.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)
    filtered = np.zeros_like(combined)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered[labels == lbl] = True

    return filtered  # bool HxW

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> np.ndarray:
    """YOLO rails overlay (red) + your color/size filtered regions (neon green)."""
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    if res.masks is None:
        return img_bgr  # nothing detected

    # Union of all rail masks at model resolution → resize to frame
    m = res.masks.data  # [N, h_m, w_m] tensor
    union = (m.sum(dim=0) > 0).float().cpu().numpy()

    H, W = img_bgr.shape[:2]
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # --- Your filter applied on the rails only (fast) ---
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        target_colors_rgb=TARGET_COLORS_RGB,
        tolerance=TOLERANCE,
        min_region_size=MIN_REGION_SIZE,
        min_region_height=MIN_REGION_HEIGHT
    )

    # --- Compose visualization ---
    out = img_bgr.copy()

    # 1) red tint for all rails
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)

    # 2) neon-green hard paint for filtered regions on top
    out[filtered_mask] = (0, 255, 0)

    return out

# =======================
# Example: directory poll (replace with your frame source)
# =======================
if __name__ == "__main__":
    import glob, time
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    seen = set()

    while True:
        for p in sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg"))):
            if p in seen:
                continue
            img = cv2.imread(p)
            if img is None:
                continue

            out = process_frame(img)
            cv2.imshow("rails + filtered", out)
            cv2.waitKey(1)

            seen.add(p)
        time.sleep(0.02)


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs
WARNING ⚠️ NMS time limit 2.050s exceeded


2025-08-04 17:15:30.890 Python[21551:28423695] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-04 17:15:30.890 Python[21551:28423695] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [ ]:
#Optimal function below

In [ ]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    Vectorized color distance + restrict work to the rail ROI for speed.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)

    # Early exit
    if not rail_mask.any():
        return filtered_full

    # --- Crop to ROI to reduce work ---
    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # --- Vectorized color match against multiple targets ---
    # Convert RGB targets -> BGR for OpenCV
    targets_bgr = np.array([(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb],
                           dtype=np.float32)  # (K,3)

    img_f = img_roi.astype(np.float32)                    # (h,w,3)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]  # (h,w,K,3)
    dist2 = np.sum(diff * diff, axis=-1)                  # (h,w,K)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)  # (h,w)

    combined_roi = mask_roi & color_mask_roi

    # --- Connected component filtering in ROI ---
    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    # Put ROI back
    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Returns:
      - out_bgr : original with red rail mask + neon green filtered regions
      - rail_mask : boolean rail mask (for debugging if needed)
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    if res.masks is None:
        return img_bgr, np.zeros(img_bgr.shape[:2], bool)

    # Union of all rail masks → resize to frame size
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()

    H, W = img_bgr.shape[:2]
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # Apply fast color/size filter inside rails
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        target_colors_rgb=TARGET_COLORS_RGB,
        tolerance=TOLERANCE,
        min_region_size=MIN_REGION_SIZE,
        min_region_height=MIN_REGION_HEIGHT
    )

    # Compose visualization
    out = img_bgr.copy()

    # red tint for all rails
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)

    # neon-green for filtered regions (hard paint)
    out[filtered_mask] = (0, 255, 0)

    return out, rail_mask

# =======================
# Run over a folder: show side-by-side and advance on SPACE
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, _ = process_frame(img)

        # Left: original | Right: filtered+masked
        combo = np.hstack((img, vis))
        cv2.imshow("Original (Left)  |  Rails+Filtered (Right)", combo)

        # Wait for SPACE (next) or 'q' (quit)
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:        # SPACE → next image
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-04 17:22:27.525 Python[22068:28431849] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-04 17:22:27.525 Python[22068:28431849] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [ ]:
#!/usr/bin/env python3
import os
import glob
import sys
import time

import cv2
import torch
import numpy as np
from ultralytics import YOLO
from skimage.morphology import skeletonize

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR)
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float,
    min_region_size: int,
    min_region_height: int
) -> np.ndarray:
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    # ROI crop
    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1
    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # color match
    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb], np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:,:,None,:] - targets_bgr[None,None,:,:]
    dist2 = np.sum(diff*diff, axis=-1)
    color_mask = np.any(dist2 <= (tolerance*tolerance), axis=-1)

    combined = mask_roi & color_mask

    # component filter
    comp_u8 = combined.astype(np.uint8)
    n, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)
    filt = np.zeros_like(combined)
    for i in range(1,n):
        area   = stats[i,cv2.CC_STAT_AREA]
        height = stats[i,cv2.CC_STAT_HEIGHT]
        if area>=min_region_size and height>=min_region_height:
            filt[labels==i] = True

    filtered_full[y0:y1, x0:x1] = filt
    return filtered_full

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray):
    """
    Returns three things:
      - vis: original tinted with rails (red) and filtered (green)
      - rail_mask: bool mask of all rails
      - filtered_mask: bool mask of color+size filtered rail pixels
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]
    H, W = img_bgr.shape[:2]
    if res.masks is None:
        return img_bgr.copy(), np.zeros((H,W),bool), np.zeros((H,W),bool)

    # union rail masks → resize
    m = res.masks.data
    union = (m.sum(dim=0)>0).float().cpu().numpy()
    if union.shape != (H,W):
        union = cv2.resize(union, (W,H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # filter inside rail_mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB,
        TOLERANCE,
        MIN_REGION_SIZE,
        MIN_REGION_HEIGHT
    )

    # compose tinted view
    vis = img_bgr.copy()
    over = vis.copy(); over[rail_mask] = (0,0,255)
    vis = cv2.addWeighted(over, ALPHA, vis, 1-ALPHA, 0)
    vis[filtered_mask] = (0,255,0)

    return vis, rail_mask, filtered_mask

# =======================
# Main: display 3 panels
# =======================
if __name__=="__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir,"frame_*.jpg")))
    if not paths:
        print("No frames found"); sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, rail_mask, filtered_mask = process_frame(img)
        lane_mask = rail_mask & (~filtered_mask)

        # skeletonize lanes
        sk = skeletonize(lane_mask).astype(np.uint8)
        sk_vis = img.copy()
        sk_vis[sk==1] = (255,255,0)  # BGR aqua

        # top row: original | tinted
        top = np.hstack((img, vis))

        # bottom: skeleton overlay spanning both halves
        blank = np.zeros_like(img)
        bot   = np.hstack((sk_vis, blank))

        canvas = np.vstack((top, bot))
        cv2.imshow("Orig | Rails+Filt  (top)   Skeleton (bottom left)", canvas)

        # advance on SPACE, quit on 'q'
        while True:
            k = cv2.waitKey(0) & 0xFF
            if k==32:    # space
                break
            if k==ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()
        time.sleep(0.02)


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-04 17:40:01.491 Python[22735:28448242] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-04 17:40:01.491 Python[22735:28448242] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [1]:
#Height filtering

In [ ]:
#!/usr/bin/env python3
import os
import glob
import sys
import time

import cv2
import torch
import numpy as np
from ultralytics import YOLO
from skimage.morphology import skeletonize

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR)
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# Skeleton pruning
MIN_SKELETON_LEN = 1000        # minimum skeleton‐pixel count to keep a branch

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float,
    min_region_size: int,
    min_region_height: int
) -> np.ndarray:
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    # ROI crop
    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1
    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # color match
    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb], np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:,:,None,:] - targets_bgr[None,None,:,:]
    dist2 = np.sum(diff*diff, axis=-1)
    color_mask = np.any(dist2 <= (tolerance*tolerance), axis=-1)

    combined = mask_roi & color_mask

    # component filter
    comp_u8 = combined.astype(np.uint8)
    n, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)
    filt = np.zeros_like(combined)
    for i in range(1,n):
        area   = stats[i,cv2.CC_STAT_AREA]
        height = stats[i,cv2.CC_STAT_HEIGHT]
        if area>=min_region_size and height>=min_region_height:
            filt[labels==i] = True

    filtered_full[y0:y1, x0:x1] = filt
    return filtered_full

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray):
    """
    Returns:
      - vis: original tinted with rails (red) and filtered (green)
      - rail_mask: bool mask of all rails
      - filtered_mask: bool mask of color+size filtered rail pixels
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]
    H, W = img_bgr.shape[:2]
    if res.masks is None:
        return img_bgr.copy(), np.zeros((H,W),bool), np.zeros((H,W),bool)

    # union rail masks → resize
    m = res.masks.data
    union = (m.sum(dim=0)>0).float().cpu().numpy()
    if union.shape != (H,W):
        union = cv2.resize(union, (W,H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # filter inside rail_mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB,
        TOLERANCE,
        MIN_REGION_SIZE,
        MIN_REGION_HEIGHT
    )

    # compose tinted view
    vis = img_bgr.copy()
    over = vis.copy(); over[rail_mask] = (0,0,255)
    vis = cv2.addWeighted(over, ALPHA, vis, 1-ALPHA, 0)
    vis[filtered_mask] = (0,255,0)

    return vis, rail_mask, filtered_mask

# =======================
# Main: display 3 panels with cleaned skeleton
# =======================
if __name__=="__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir,"frame_*.jpg")))
    if not paths:
        print("No frames found"); sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, rail_mask, filtered_mask = process_frame(img)
        lane_mask = rail_mask & (~filtered_mask)

        # skeletonize lanes
        sk = skeletonize(lane_mask).astype(np.uint8)

        # prune short skeleton branches
        n_s, lbls = cv2.connectedComponents(sk, connectivity=8)
        sk_clean = np.zeros_like(sk)
        for lbl in range(1, n_s):
            if (lbls==lbl).sum() >= MIN_SKELETON_LEN:
                sk_clean[lbls==lbl] = 1

        # make skeleton overlay
        sk_vis = img.copy()
        sk_vis[sk_clean==1] = (255,255,0)  # BGR aqua

        # top row: original | tinted+filtered
        top = np.hstack((img, vis))

        # bottom: cleaned skeleton overlay | blank
        blank = np.zeros_like(img)
        bottom = np.hstack((sk_vis, blank))

        canvas = np.vstack((top, bottom))
        cv2.imshow("Orig | Rails+Filt  (top)   Skeleton (bottom left)", canvas)

        # advance on SPACE, quit on 'q'
        while True:
            k = cv2.waitKey(0) & 0xFF
            if k == 32:    # space
                break
            if k == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()
        time.sleep(0.02)


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-04 17:56:01.634 Python[23543:28466670] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-04 17:56:01.634 Python[23543:28466670] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [ ]:
#!/usr/bin/env python3
import os
import glob
import sys
import time

import cv2
import torch
import numpy as np
from ultralytics import YOLO
from skimage.morphology import skeletonize

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR)
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# Skeleton pruning by width
MIN_SKELETON_WIDTH = 60       # minimum lane-width in px to keep a skeleton branch

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float,
    min_region_size: int,
    min_region_height: int
) -> np.ndarray:
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    # ROI crop
    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1
    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # color match
    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb], np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:,:,None,:] - targets_bgr[None,None,:,:]
    dist2 = np.sum(diff*diff, axis=-1)
    color_mask = np.any(dist2 <= (tolerance*tolerance), axis=-1)

    combined = mask_roi & color_mask

    # component filter
    comp_u8 = combined.astype(np.uint8)
    n, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)
    filt = np.zeros_like(combined)
    for i in range(1,n):
        area   = stats[i,cv2.CC_STAT_AREA]
        height = stats[i,cv2.CC_STAT_HEIGHT]
        if area>=min_region_size and height>=min_region_height:
            filt[labels==i] = True

    filtered_full[y0:y1, x0:x1] = filt
    return filtered_full

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray):
    """
    Returns:
      - vis: original tinted with rails (red) and filtered (green)
      - rail_mask: bool mask of all rails
      - filtered_mask: bool mask of color+size filtered rail pixels
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]
    H, W = img_bgr.shape[:2]
    if res.masks is None:
        return img_bgr.copy(), np.zeros((H,W),bool), np.zeros((H,W),bool)

    # union rail masks → resize
    m = res.masks.data
    union = (m.sum(dim=0)>0).float().cpu().numpy()
    if union.shape != (H,W):
        union = cv2.resize(union, (W,H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # filter inside rail_mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB,
        TOLERANCE,
        MIN_REGION_SIZE,
        MIN_REGION_HEIGHT
    )

    # compose tinted view
    vis = img_bgr.copy()
    over = vis.copy(); over[rail_mask] = (0,0,255)
    vis = cv2.addWeighted(over, ALPHA, vis, 1-ALPHA, 0)
    vis[filtered_mask] = (0,255,0)

    return vis, rail_mask, filtered_mask

# =======================
# Main: display 3 panels with width-pruned skeleton
# =======================
if __name__=="__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir,"frame_*.jpg")))
    if not paths:
        print("No frames found"); sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, rail_mask, filtered_mask = process_frame(img)
        lane_mask = rail_mask & (~filtered_mask)

        # skeletonize lanes
        sk = skeletonize(lane_mask).astype(np.uint8)

        # compute distance‐transform of lane_mask
        dist = cv2.distanceTransform((lane_mask*255).astype(np.uint8),
                                     cv2.DIST_L2, cv2.DIST_MASK_PRECISE)

        # prune skeleton branches narrower than MIN_SKELETON_WIDTH
        n_s, lbls = cv2.connectedComponents(sk, connectivity=8)
        sk_clean = np.zeros_like(sk)
        for lbl in range(1, n_s):
            comp = (lbls == lbl)
            # measure maximum width along this branch
            # width ≈ 2 * max(distance_transform)
            max_radius = dist[comp].max() if comp.any() else 0
            max_width = 2 * max_radius
            if max_width >= MIN_SKELETON_WIDTH:
                sk_clean[comp] = 1

        # make skeleton overlay
        sk_vis = img.copy()
        sk_vis[sk_clean==1] = (255,255,0)  # BGR aqua

        # top row: original | tinted+filtered
        top = np.hstack((img, vis))

        # bottom: width-pruned skeleton overlay | blank
        blank = np.zeros_like(img)
        bottom = np.hstack((sk_vis, blank))

        canvas = np.vstack((top, bottom))
        cv2.imshow("Orig | Rails+Filt (top)   Skeleton (bottom left)", canvas)

        # advance on SPACE, quit on 'q'
        while True:
            k = cv2.waitKey(0) & 0xFF
            if k == 32:    # space
                break
            if k == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()
        time.sleep(0.02)


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-04 18:00:32.147 Python[23910:28473671] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-04 18:00:32.147 Python[23910:28473671] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
